In [22]:
#!/usr/bin/env python3

import pandas as pd
import errno    
import os
import numpy as np
import math

from sklearn.metrics import r2_score
from scipy import stats

from sklearn.metrics import mean_squared_error

'''
Author: Enrico Ceccolini
    Calculate the mean core power consumption (core_type1, core_type2 and core type_3)
    by selecting jobs that completely ran on one of this types
'''

#datadir = "/datasets/eurora_data/db/"
datadir = "/datasets/eurora_data/db1/"

### select an interval from
## 1 settings wholeData
interval_comment = "WholeData"
suffix = "_1min_"

### select an interval from
## 2 settings Andrea
interval_comment = "Andrea"
andrea_start_time = pd.to_datetime('2014-03-31')
andrea_end_time = pd.to_datetime('2014-05-01')

## 3 settings Alina
#interval_comment = "Alina"
#alina_start_time = pd.to_datetime('2014-09-30')
#alina_end_time = pd.to_datetime('2014-11-01')


infile_long_jobs = datadir + "CPUs/" + interval_comment + "/long_jobs_real_pow_" + interval_comment + ".csv"
interval_long_jobs = pd.read_csv(infile_long_jobs, index_col=0)

infile_short_jobs = datadir + "CPUs/" + interval_comment + "/short_jobs_real_pow_" + interval_comment + ".csv"
interval_short_jobs = pd.read_csv(infile_short_jobs, index_col=0)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


nodes 1-16 and 25-32 have a maximum frequency of 2.1GHz (CPUs-2100), nodes 17-24 have a maximum frequency of 2.2GHz (CPUs-2200), and nodes 33-64 have a maximum frequency of 3.1GHz (CPUs-3100). For the accelerators, nodes from 0 to 32 embed Intel Xeon Phi accelerators, whereas nodes from 33 to 64 embed Nvidia Kepler GPUs.

In [23]:
interval_long_jobs.head(1)

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,exit_status,pbs_exit_code,gpu_req,mic_req,real_pow,runned_alone,real_pow_quality,n_2_1,n_2_2,n_3_1
job_id,,,,,,,,,,,,,,,,,,,,,
1,498458.node129,rth2e6t0,parallel,2014-03-31 00:30:02,2014-03-31 00:30:02,2014-03-31 01:22:40,alupi000@node129.eurora.cineca.it,1,16,14,...,COMPLETED,NaN,NaN,NaN,140.0,True,0.0,0,1,0


In [24]:
print("There are {} long jobs in the {}'s interval".format(interval_long_jobs.shape[0], interval_comment))
print("There are {} short jobs in the {}'s interval".format(interval_short_jobs.shape[0], interval_comment))

There are 22358 long jobs in the Andrea's interval
There are 66179 short jobs in the Andrea's interval


$$𝑁𝑀𝑆𝐸=1−||𝑥−𝑦||2||𝑥−𝑥¯||$$

In [47]:
interval_long_jobs_2_1 = interval_long_jobs[interval_long_jobs['node_req'] == interval_long_jobs['n_2_1']]
print("There are {} jobs completely running on this type of cpu".format(interval_long_jobs_2_1.shape[0]))
mean_core_2_1 = interval_long_jobs_2_1['real_pow'].sum() / interval_long_jobs_2_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_1))
interval_long_jobs_2_1['predicted'] = mean_core_2_1 * interval_long_jobs_2_1['cpu_req']
#print("Accuracy: {}".format(r2_score(interval_long_jobs_2_1['real_pow'], interval_long_jobs_2_1['predicted'])))
slope, intercept, r_value, p_value, std_err = stats.linregress(interval_long_jobs_2_1['real_pow'], interval_long_jobs_2_1['predicted'])
print("R-squared: {}".format(r_value**2))
print("mean_quared_error: {}".format(mean_squared_error(interval_long_jobs_2_1['real_pow'], interval_long_jobs_2_1['predicted'])))

mean_P = interval_long_jobs_2_1['real_pow'].mean()
#mean_M = interval_long_jobs_2_1['predicted'].mean()
interval_long_jobs_2_1['NMSE'] = ((interval_long_jobs_2_1['real_pow'] - interval_long_jobs_2_1['predicted'])**2 / (mean_P*mean_M))

There are 5047 jobs completely running on this type of cpu
The average per core is 12.441707070404295 W
R-squared: 0.7748428780751343
mean_quared_error: 3099.717591810004


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [48]:
interval_long_jobs_2_1['NMSE'].sum() / interval_long_jobs_2_1.shape[0]

0.18191540922735214

### Compute the mean core power consumption for the first type of cpu (2.1 GHz) 

In [44]:
interval_long_jobs_2_1 = interval_long_jobs[interval_long_jobs['node_req'] == interval_long_jobs['n_2_1']]
print("There are {} jobs completely running on this type of cpu".format(interval_long_jobs_2_1.shape[0]))
mean_core_2_1 = interval_long_jobs_2_1['real_pow'].sum() / interval_long_jobs_2_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_1))
interval_long_jobs_2_1['predicted'] = mean_core_2_1 * interval_long_jobs_2_1['cpu_req']
#print("Accuracy: {}".format(r2_score(interval_long_jobs_2_1['real_pow'], interval_long_jobs_2_1['predicted'])))
slope, intercept, r_value, p_value, std_err = stats.linregress(interval_long_jobs_2_1['real_pow'], interval_long_jobs_2_1['predicted'])
print("R-squared: {}".format(r_value**2))
print("mean_quared_error: {}".format(mean_squared_error(interval_long_jobs_2_1['real_pow'], interval_long_jobs_2_1['predicted'])))
mean_P = interval_long_jobs_2_1['real_pow'].mean()
mean_M = interval_long_jobs_2_1['predicted'].mean()
interval_long_jobs_2_1['NMSE'] = ((interval_long_jobs_2_1['real_pow'] - interval_long_jobs_2_1['predicted'])**2 / (mean_P*mean_M))
print("NMSE: {}".format(interval_long_jobs_2_1['NMSE'].sum() / interval_long_jobs_2_1.shape[0]))

### Esclude the jobs with 0 of approximation
quality_long_jobs_2_1 = interval_long_jobs_2_1[interval_long_jobs_2_1['real_pow'] != 0]
quality_long_jobs_2_1 = quality_long_jobs_2_1[quality_long_jobs_2_1['real_pow_quality'] != 0]
print("There are {} jobs completely running on this type of cpu with that quality".format(quality_long_jobs_2_1.shape[0]))
mean_core_2_1 = quality_long_jobs_2_1['real_pow'].sum() / quality_long_jobs_2_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_1))
quality_long_jobs_2_1['predicted'] = mean_core_2_1 * quality_long_jobs_2_1['cpu_req']
#print("Accuracy: {}".format(r2_score(quality_long_jobs_2_1['real_pow'], quality_long_jobs_2_1['predicted'])))
slope, intercept, r_value, p_value, std_err = stats.linregress(quality_long_jobs_2_1['real_pow'], quality_long_jobs_2_1['predicted'])
print("R-squared: {}".format(r_value**2))
print("mean_quared_error: {}".format(mean_squared_error(quality_long_jobs_2_1['real_pow'], quality_long_jobs_2_1['predicted'])))
mean_P = quality_long_jobs_2_1['real_pow'].mean()
mean_M = quality_long_jobs_2_1['predicted'].mean()
quality_long_jobs_2_1['NMSE'] = ((quality_long_jobs_2_1['real_pow'] - quality_long_jobs_2_1['predicted'])**2 / (mean_P*mean_M))
print("NMSE: {}".format(quality_long_jobs_2_1['NMSE'].sum() / quality_long_jobs_2_1.shape[0]))



### Esclude the jobs with low quality of approximation
quality_long_jobs_2_1 = interval_long_jobs_2_1[interval_long_jobs_2_1['real_pow'] != 0]
quality_long_jobs_2_1 = quality_long_jobs_2_1[quality_long_jobs_2_1['real_pow_quality'] != 0]
quality_long_jobs_2_1 = quality_long_jobs_2_1[quality_long_jobs_2_1['real_pow_quality'] > 0.5]
print("There are {} jobs completely running on this type of cpu with that quality".format(quality_long_jobs_2_1.shape[0]))
mean_core_2_1 = quality_long_jobs_2_1['real_pow'].sum() / quality_long_jobs_2_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_1))
quality_long_jobs_2_1['predicted'] = mean_core_2_1 * quality_long_jobs_2_1['cpu_req']
#print("Accuracy: {}".format(r2_score(quality_long_jobs_2_1['real_pow'], quality_long_jobs_2_1['predicted'])))
slope, intercept, r_value, p_value, std_err = stats.linregress(quality_long_jobs_2_1['real_pow'], quality_long_jobs_2_1['predicted'])
print("R-squared: {}".format(r_value**2))
print("mean_quared_error: {}".format(mean_squared_error(quality_long_jobs_2_1['real_pow'], quality_long_jobs_2_1['predicted'])))
mean_P = quality_long_jobs_2_1['real_pow'].mean()
mean_M = quality_long_jobs_2_1['predicted'].mean()
quality_long_jobs_2_1['NMSE'] = ((quality_long_jobs_2_1['real_pow'] - quality_long_jobs_2_1['predicted'])**2 / (mean_P*mean_M))
print("NMSE: {}".format(quality_long_jobs_2_1['NMSE'].sum() / quality_long_jobs_2_1.shape[0]))   
    
    
    
quality_long_jobs_2_1 = interval_long_jobs_2_1[interval_long_jobs_2_1['real_pow'] != 0]
quality_long_jobs_2_1 = quality_long_jobs_2_1[quality_long_jobs_2_1['real_pow_quality'] != 0]
quality_long_jobs_2_1 = quality_long_jobs_2_1[quality_long_jobs_2_1['runned_alone'] == True]
print("There are {} jobs completely running on this type of cpu with that quality".format(quality_long_jobs_2_1.shape[0]))
mean_core_2_1 = quality_long_jobs_2_1['real_pow'].sum() / quality_long_jobs_2_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_1))
quality_long_jobs_2_1['predicted'] = mean_core_2_1 * quality_long_jobs_2_1['cpu_req']
slope, intercept, r_value, p_value, std_err = stats.linregress(quality_long_jobs_2_1['real_pow'], quality_long_jobs_2_1['predicted'])
print("R-squared: {}".format(r_value**2))
print("mean_quared_error: {}".format(mean_squared_error(quality_long_jobs_2_1['real_pow'], quality_long_jobs_2_1['predicted'])))
mean_P = quality_long_jobs_2_1['real_pow'].mean()
mean_M = quality_long_jobs_2_1['predicted'].mean()
quality_long_jobs_2_1['NMSE'] = ((quality_long_jobs_2_1['real_pow'] - quality_long_jobs_2_1['predicted'])**2 / (mean_P*mean_M))
print("NMSE: {}".format(quality_long_jobs_2_1['NMSE'].sum() / quality_long_jobs_2_1.shape[0]))

There are 5047 jobs completely running on this type of cpu
The average per core is 12.441707070404295 W
R-squared: 0.7748428780751343
mean_quared_error: 3099.717591810004
NMSE: 0.226345587521094
There are 4984 jobs completely running on this type of cpu with that quality
The average per core is 12.482594115975424 W


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


R-squared: 0.7745453387119718
mean_quared_error: 3137.302363842254
NMSE: 0.22688879692086222
There are 4915 jobs completely running on this type of cpu with that quality
The average per core is 12.461791461036658 W
R-squared: 0.776326504924278
mean_quared_error: 3126.8401950196994
NMSE: 0.22475655128959907
There are 3319 jobs completely running on this type of cpu with that quality
The average per core is 11.91715441740345 W
R-squared: 0.732600394714791
mean_quared_error: 4012.7634718153327
NMSE: 0.18927287381707084


In [23]:
quality_long_jobs_2_1['predicted'] = 12.188997397054441 * quality_long_jobs_2_1['cpu_req']
print("Accuracy: {}".format(r2_score(quality_long_jobs_2_1['real_pow'], quality_long_jobs_2_1['predicted'])))

Accuracy: 0.6961530048060831


In [25]:
#quality_long_jobs_2_1

### Andrea's interval

In [10]:
andrea_interval_long_jobs =interval_long_jobs[pd.to_datetime(interval_long_jobs['run_start_time']) >= andrea_start_time]
andrea_interval_long_jobs =andrea_interval_long_jobs[pd.to_datetime(andrea_interval_long_jobs['run_start_time']) <= andrea_end_time]

In [26]:
interval_long_jobs_2_1 = andrea_interval_long_jobs[andrea_interval_long_jobs['node_req'] == andrea_interval_long_jobs['n_2_1']]
print("There are {} jobs completely running on this type of cpu".format(interval_long_jobs_2_1.shape[0]))
mean_core_2_1 = interval_long_jobs_2_1['real_pow'].sum() / interval_long_jobs_2_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_1))
interval_long_jobs_2_1['predicted'] = mean_core_2_1 * interval_long_jobs_2_1['cpu_req']
#print("Accuracy: {}".format(r2_score(interval_long_jobs_2_1['real_pow'], interval_long_jobs_2_1['predicted'])))
slope, intercept, r_value, p_value, std_err = stats.linregress(interval_long_jobs_2_1['real_pow'], interval_long_jobs_2_1['predicted'])
print("R-squared: {}".format(r_value**2))

### Esclude the jobs with low quality of approximation
quality_long_jobs_2_1 = interval_long_jobs_2_1[interval_long_jobs_2_1['real_pow'] != 0]
quality_long_jobs_2_1 = quality_long_jobs_2_1[quality_long_jobs_2_1['real_pow_quality'] != 0]
print("There are {} jobs completely running on this type of cpu with that quality".format(quality_long_jobs_2_1.shape[0]))
mean_core_2_1 = quality_long_jobs_2_1['real_pow'].sum() / quality_long_jobs_2_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_1))
quality_long_jobs_2_1['predicted'] = mean_core_2_1 * quality_long_jobs_2_1['cpu_req']
#print("Accuracy: {}".format(r2_score(quality_long_jobs_2_1['real_pow'], quality_long_jobs_2_1['predicted'])))
slope, intercept, r_value, p_value, std_err = stats.linregress(quality_long_jobs_2_1['real_pow'], quality_long_jobs_2_1['predicted'])
print("R-squared: {}".format(r_value**2))


### Esclude jobs that ran concurrently
#quality_long_jobs_2_1 = interval_long_jobs_2_1[interval_long_jobs_2_1['real_pow'] != 0]
#quality_long_jobs_2_1 = quality_long_jobs_2_1[quality_long_jobs_2_1['real_pow_quality'] != 0]
#quality_long_jobs_2_1 = quality_long_jobs_2_1[quality_long_jobs_2_1['runned_alone'] == True]
#print("There are {} jobs completely running on this type of cpu with that quality".format(quality_long_jobs_2_1.shape[0]))
#mean_core_2_1 = quality_long_jobs_2_1['real_pow'].sum() / quality_long_jobs_2_1['cpu_req'].sum()
#print("The average per core is {} W".format(mean_core_2_1))
#quality_long_jobs_2_1['predicted'] = mean_core_2_1 * quality_long_jobs_2_1['cpu_req']
#print("Accuracy: {}".format(r2_score(quality_long_jobs_2_1['real_pow'], quality_long_jobs_2_1['predicted'])))

There are 5064 jobs completely running on this type of cpu
The average per core is 12.143704579416486 W
R-squared: 0.7233019844855627
There are 4900 jobs completely running on this type of cpu with that quality
The average per core is 12.529485612237833 W
R-squared: 0.7673655375680561


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


### Compute the mean core power consumption for the first type of cpu (2.2 GHz) 

In [27]:
interval_long_jobs_2_2 = interval_long_jobs[interval_long_jobs['node_req'] == interval_long_jobs['n_2_2']]
print("There are {} jobs completely running on this type of cpu".format(interval_long_jobs_2_2.shape[0]))
mean_core_2_2 = interval_long_jobs_2_2['real_pow'].sum() / interval_long_jobs_2_2['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_2))
interval_long_jobs_2_2['predicted'] = mean_core_2_2 * interval_long_jobs_2_2['cpu_req']
#print("Accuracy: {}".format(r2_score(interval_long_jobs_2_2['real_pow'], interval_long_jobs_2_2['predicted'])))
slope, intercept, r_value, p_value, std_err = stats.linregress(interval_long_jobs_2_2['real_pow'], interval_long_jobs_2_2['predicted'])
print("R-squared: {}".format(r_value**2))

quality_long_jobs_2_2 = interval_long_jobs_2_2[interval_long_jobs_2_2['real_pow'] != 0]
quality_long_jobs_2_2 = quality_long_jobs_2_2[quality_long_jobs_2_2['real_pow_quality'] != 0]
print("There are {} jobs completely running on this type of cpu with that quality".format(quality_long_jobs_2_2.shape[0]))
mean_core_2_2 = quality_long_jobs_2_2['real_pow'].sum() / quality_long_jobs_2_2['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_2))
quality_long_jobs_2_2['predicted'] = mean_core_2_2 * quality_long_jobs_2_2['cpu_req']
#print("Accuracy: {}".format(r2_score(quality_long_jobs_2_2['real_pow'], quality_long_jobs_2_2['predicted'])))
slope, intercept, r_value, p_value, std_err = stats.linregress(quality_long_jobs_2_2['real_pow'], quality_long_jobs_2_2['predicted'])
print("R-squared: {}".format(r_value**2))

There are 4570 jobs completely running on this type of cpu
The average per core is 13.80509054541727 W
R-squared: 0.48435611996160277
There are 4294 jobs completely running on this type of cpu with that quality
The average per core is 14.664222292746754 W
R-squared: 0.5748425521907099


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


### Andrea's interval

In [28]:
interval_long_jobs_2_2 = andrea_interval_long_jobs[andrea_interval_long_jobs['node_req'] == andrea_interval_long_jobs['n_2_2']]
print("There are {} jobs completely running on this type of cpu".format(interval_long_jobs_2_2.shape[0]))
mean_core_2_2 = interval_long_jobs_2_2['real_pow'].sum() / interval_long_jobs_2_2['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_2))
interval_long_jobs_2_2['predicted'] = mean_core_2_2 * interval_long_jobs_2_2['cpu_req']
#print("Accuracy: {}".format(r2_score(interval_long_jobs_2_2['real_pow'], interval_long_jobs_2_2['predicted'])))
slope, intercept, r_value, p_value, std_err = stats.linregress(interval_long_jobs_2_2['real_pow'], interval_long_jobs_2_2['predicted'])
print("R-squared: {}".format(r_value**2))


quality_long_jobs_2_2 = interval_long_jobs_2_2[interval_long_jobs_2_2['real_pow'] != 0]
quality_long_jobs_2_2 = quality_long_jobs_2_2[quality_long_jobs_2_2['real_pow_quality'] != 0]
print("There are {} jobs completely running on this type of cpu with that quality".format(quality_long_jobs_2_2.shape[0]))
mean_core_2_2 = quality_long_jobs_2_2['real_pow'].sum() / quality_long_jobs_2_2['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_2))
quality_long_jobs_2_2['predicted'] = mean_core_2_2 * quality_long_jobs_2_2['cpu_req']
#print("Accuracy: {}".format(r2_score(quality_long_jobs_2_2['real_pow'], quality_long_jobs_2_2['predicted'])))
slope, intercept, r_value, p_value, std_err = stats.linregress(quality_long_jobs_2_2['real_pow'], quality_long_jobs_2_2['predicted'])
print("R-squared: {}".format(r_value**2))

There are 1428 jobs completely running on this type of cpu
The average per core is 14.932216042263958 W
R-squared: 0.5989269638151288
There are 1362 jobs completely running on this type of cpu with that quality
The average per core is 15.45302698203022 W
R-squared: 0.6332755598899138


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


### Compute the mean core power consumption for the first type of cpu (3.1 GHz) 

In [29]:
interval_long_jobs_3_1 = interval_long_jobs[interval_long_jobs['node_req'] == interval_long_jobs['n_3_1']]
print("There are {} jobs completely running on this type of cpu".format(interval_long_jobs_3_1.shape[0]))
mean_core_3_1 = interval_long_jobs_3_1['real_pow'].sum() / interval_long_jobs_3_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_3_1))
interval_long_jobs_3_1['predicted'] = mean_core_3_1 * interval_long_jobs_3_1['cpu_req']
#print("Accuracy: {}".format(r2_score(interval_long_jobs_3_1['real_pow'], interval_long_jobs_3_1['predicted'])))
slope, intercept, r_value, p_value, std_err = stats.linregress(interval_long_jobs_3_1['real_pow'], interval_long_jobs_3_1['predicted'])
print("R-squared: {}".format(r_value**2))


quality_long_jobs_3_1 = interval_long_jobs_3_1[interval_long_jobs_3_1['real_pow'] != 0]
quality_long_jobs_3_1 = quality_long_jobs_3_1[quality_long_jobs_3_1['real_pow_quality'] != 0]
print("There are {} jobs completely running on this type of cpu with that quality".format(quality_long_jobs_3_1.shape[0]))
mean_core_3_1 = quality_long_jobs_3_1['real_pow'].sum() / quality_long_jobs_3_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_3_1))
quality_long_jobs_3_1['predicted'] = mean_core_3_1 * quality_long_jobs_3_1['cpu_req']
#print("Accuracy: {}".format(r2_score(quality_long_jobs_3_1['real_pow'], quality_long_jobs_3_1['predicted'])))
slope, intercept, r_value, p_value, std_err = stats.linregress(quality_long_jobs_3_1['real_pow'], quality_long_jobs_3_1['predicted'])
print("R-squared: {}".format(r_value**2))

There are 43708 jobs completely running on this type of cpu
The average per core is 24.842150667100178 W
R-squared: 0.7177960495658504
There are 38419 jobs completely running on this type of cpu with that quality
The average per core is 27.562140558509466 W
R-squared: 0.7925790456244505


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


### Andrea's interval

In [30]:
interval_long_jobs_3_1 = andrea_interval_long_jobs[andrea_interval_long_jobs['node_req'] == andrea_interval_long_jobs['n_3_1']]
print("There are {} jobs completely running on this type of cpu".format(interval_long_jobs_3_1.shape[0]))
mean_core_3_1 = interval_long_jobs_3_1['real_pow'].sum() / interval_long_jobs_3_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_3_1))
interval_long_jobs_3_1['predicted'] = mean_core_3_1 * interval_long_jobs_3_1['cpu_req']
#print("Accuracy: {}".format(r2_score(interval_long_jobs_3_1['real_pow'], interval_long_jobs_3_1['predicted'])))
slope, intercept, r_value, p_value, std_err = stats.linregress(interval_long_jobs_3_1['real_pow'], interval_long_jobs_3_1['predicted'])
print("R-squared: {}".format(r_value**2))

quality_long_jobs_3_1 = interval_long_jobs_3_1[interval_long_jobs_3_1['real_pow'] != 0]
quality_long_jobs_3_1 = quality_long_jobs_3_1[quality_long_jobs_3_1['real_pow_quality'] != 0]
print("There are {} jobs completely running on this type of cpu with that quality".format(quality_long_jobs_3_1.shape[0]))
mean_core_3_1 = quality_long_jobs_3_1['real_pow'].sum() / quality_long_jobs_3_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_3_1))
quality_long_jobs_3_1['predicted'] = mean_core_3_1 * quality_long_jobs_3_1['cpu_req']
#print("Accuracy: {}".format(r2_score(quality_long_jobs_3_1['real_pow'], quality_long_jobs_3_1['predicted'])))
slope, intercept, r_value, p_value, std_err = stats.linregress(quality_long_jobs_3_1['real_pow'], quality_long_jobs_3_1['predicted'])
print("R-squared: {}".format(r_value**2))

There are 15587 jobs completely running on this type of cpu
The average per core is 22.25328132443491 W
R-squared: 0.7079937136710611
There are 13793 jobs completely running on this type of cpu with that quality
The average per core is 25.079102763361274 W
R-squared: 0.8222648015787352


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


### Debug

In [32]:
interval_long_jobs = interval_long_jobs[interval_long_jobs['real_pow_quality'] != 0]
interval_long_jobs.shape

(59542, 19)

In [33]:
interval_long_jobs = interval_long_jobs[interval_long_jobs['real_pow'] != 0]
interval_long_jobs.shape

(57658, 19)

In [35]:
interval_long_jobs_2_1 = interval_long_jobs[interval_long_jobs['node_req'] == interval_long_jobs['n_2_1']]

In [36]:
interval_long_jobs_2_1.shape

(12680, 19)

In [37]:
mean_core_2_1 = interval_long_jobs_2_1['real_pow'].sum() / interval_long_jobs_2_1['cpu_req'].sum()
print(mean_core_2_1)

In [39]:
interval_long_jobs_2_1['predicted'] = mean_core_2_1 * interval_long_jobs_2_1['cpu_req']

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
#interval_long_jobs_2_1

In [41]:
print("approximation accuracy: {}".format(r2_score(interval_long_jobs_2_1['real_pow'], interval_long_jobs_2_1['predicted'])))

approximation accuracy: 0.6961530048060831


(12327, 20)

In [44]:
interval_long_jobs_2_1_top_quality = interval_long_jobs_2_1[interval_long_jobs_2_1['real_pow_quality'] == 1]
interval_long_jobs_2_1_top_quality.shape
mean_core_2_1_top_quality = interval_long_jobs_2_1['real_pow'].sum() / interval_long_jobs_2_1['cpu_req'].sum()
print(mean_core_2_1_top_quality)
interval_long_jobs_2_1_top_quality['predicted'] = mean_core_2_1_top_quality * interval_long_jobs_2_1_top_quality['cpu_req']
print("approximation accuracy: {}".format(r2_score(interval_long_jobs_2_1_top_quality['real_pow'], interval_long_jobs_2_1_top_quality['predicted'])))

12.188997397054441
approximation accuracy: 0.6984105718416773


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
